In [9]:
import numpy as np 
import pandas as pd
import sys

dataX = "/home/adkishor/projects/misc/CSCI561-AI/mlp/publicdata/train_image.csv"#sys.argv[1]
dataY = "/home/adkishor/projects/misc/CSCI561-AI/mlp/publicdata/train_label.csv"#sys.argv[2]

TEST = "/home/adkishor/projects/misc/CSCI561-AI/mlp/publicdata/test_image.csv"#sys.argv[3]

def load_data(X,Y, T):

    train_data = pd.read_csv(X, header=None)
    train_labels = pd.read_csv(Y, header=None)
    test_data = pd.read_csv(T, header=None)
    return train_data, train_labels, test_data

first , second, third = load_data(dataX, dataY, TEST)
print(first.shape , second.shape, third.shape)





(60000, 784) (60000, 1) (10000, 784)


In [10]:
def sigmoid(X):
    z =  1/(1+np.exp(-X))
    return z

In [11]:

def softmax(X):
    e = np.exp(X)
    e = e/e.sum()
    
    return e

In [12]:
X = np.array(first.T)
Y = np.array(second.T)
x_hat = np.array(third.T)

In [13]:
Yonehat = np.zeros((X.shape[1], 10))
for i in range(X.shape[1]):
    Yonehat[i][second[0][i]] = 1

In [14]:
Yonehat = Yonehat.T

X.shape, Y.shape, x_hat.shape, Yonehat.shape

((784, 60000), (1, 60000), (784, 10000), (10, 60000))

In [ ]:
batch_size = 32
learning_rate = 0.001
epochs = 50

params = {
    'w1':np.random.randn(512, 784),
    'b1':np.random.randn(512,1),
    'w2':np.random.randn(256, 512),
    'b2':np.random.randn(256,1),
    'w3':np.random.randn(10,256),
    'b3':np.random.randn(10,1)
}


def cross_entropy(Y, out):
    m = 